<a href="https://colab.research.google.com/github/cras-lab/OpenAPI/blob/main/BOK_%EC%88%98%EC%8B%A0%EA%B8%88%EB%A6%AC%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요한 모듈을 임포트 한다.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

한국은행 API에 들어갈 매개변수를 설정한다.

In [ ]:
# API configuration
API_KEY = "2NBFHJY9F0DBETZYMFLB"  # 본인의 키를 설정한다.
START_DATE = "201801"   # 기준금리 읽을 시작 월
END_DATE = "202506"     # 기준금리 읽을 마지막 월

한국은행의 해당 OpenAPI call URL을 설정한다.

In [ ]:
url = f"https://ecos.bok.or.kr/api/StatisticSearch/{API_KEY}/json/kr/1/1000/121Y002/M/{START_DATE}/{END_DATE}/"

실제 값을 읽어 온다.

In [ ]:
# Request
response = requests.get(url)
data = response.json()

오류면 중단하고, 값을 제대로 읽어 왔으면 data frame으로 변환한다.

In [ ]:
# Check and convert to DataFrame
if 'StatisticSearch' not in data:
    import json
    print("❌ Error:\n", json.dumps(data, indent=2, ensure_ascii=False))
    raise SystemExit

df = pd.DataFrame(data['StatisticSearch']['row'])

데이터 중 기준금리 부분만 추출한다.

In [ ]:
# Filter only for "한국은행 기준금리"
df = df[df['ITEM_NAME1'] == '저축성수신'].copy()

시간형식에 맞게 변환

In [ ]:
# Convert TIME to datetime and DATA_VALUE to float
df['TIME'] = pd.to_datetime(df['TIME'], format='%Y%m')
df['DATA_VALUE'] = pd.to_numeric(df['DATA_VALUE'], errors='coerce')

# Clean
df = df.dropna(subset=['TIME', 'DATA_VALUE']).sort_values('TIME')

그래프로 도식화 해 본다.

In [ ]:
# Plot
plt.figure(figsize=(12, 6))
plt.plot(df['TIME'], df['DATA_VALUE'], marker='o', linestyle='-')
plt.title("Deposit Rates  (BOK OpenAPI)", fontsize=14)
plt.xlabel("Period")
plt.ylabel(" Deposit Rate (%)")
plt.grid(True)
plt.tight_layout()
plt.show()
